In [14]:
curl https://sh.rustup.rs -sSf | sh

SyntaxError: invalid syntax (<ipython-input-14-96a07a4feb3d>, line 1)

In [8]:
import sys
!{sys.executable} -m pip install transformers==3.0.0

  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached tokenizers-0.8.0rc4-cp36-cp36m-macosx_10_10_x86_64.whl (2.1 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
     |████████████████████████████████| 1.2 MB 3.9 MB/s eta 0:00:01
  Using cached regex-2022.1.18-cp36-cp36m-macosx_10_9_x86_64.whl (289 kB)


In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import itertools

AttributeError: module 'numpy' has no attribute '__version__'

In [11]:
#chunk data by statements to avoid RAM overflow
print("Load data")
all_data = pd.read_csv("/Users/abhijit/IGRI/data/step1_data_sample.csv")
statements = all_data.groupby('sid')
sid = all_data['sid'].unique()
print(sid)
all_features = []

Load data
[324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341
 343 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361
 362 363 364 365 366 367 368 379 380 381 382 383 384 385 386 387 388 389
 390 391 392 393 394 395 396 397 398 399 400 401 402 403 404 410 411 412
 413 414 415 416 417 418 419 420 421 422 423 424 425 427 428 429 430 431
 432 433 435 437 438 439 440 441 442 443 444 445 446 447 448 449 450 451
 452 453 454 455 456 457 459 460 461 462 463 464 465 466 467 468 469 470
 471 472 473 474 475 476 477 478 479 480 481 482 483 484 485 486 487 488
 489 490 491 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506
 507 508 509 510 513 514 515 516 517 518 519 520 521 523 524 526 528 529
 531 533 534 536 538 540 542 544 545 547 548 550 551 552 553 554 555 556
 557 558 559 560 561 562 563 564 565 566 567 568 569 570 571 572 573 574
 579 580 581 582 583 584 585 586 587 588 589 590 591 592 593 594 596 597
 598 599 600 601 602 603 604 605 606 607 

In [12]:
# Pretrained model
print("Load model")
# For DistilBERT:
print( "Model: distilbert-base-uncased")
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

Load model
Model: distilbert-base-uncased


In [13]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [19]:
for statement_no in sid:
  print(statement_no)
  df = statements.get_group(statement_no)
  df = df.rename(columns=dict(zip(df.columns,[0, "sid", "sid_text","tid",1,2])))
  #print(df.shape)
  #batch_1 = df[:2000].reset_index()
  ### clean up sentences to preempt tokenization problems
  ### this is all effort to make the space tokenization look more like the bert tokenization, especially special handling of ellipses and hypthens
  print(df.head(5))
  df = df.assign(sid_text=df['sid_text'].apply(
      lambda x:x.lower().replace('-lrb-', '[').replace('-rrb-', ']').replace('...', '<ELLIPSES>').replace('.', '').replace( '<ELLIPSES>', '...').replace(' - ', ' , ').replace('-', '').replace('  ', ' ').strip()
      ))
  #df = df.assign(sid_text_old=df['sid_text'])
  #df = df.assign(sid_text=df['sid_text'].apply(
      #lambda x:x.lower().replace('.', '').replace('-', '').replace('  ', ' ').replace('root ', '').strip()
      #))

  batch_1 = df
  batch_1[1].value_counts()

  print("Prep data")
  # Tokenization
  tokenized_raw = batch_1['sid_text'].apply((lambda x: tokenizer.tokenize(x)))
  tokenized = batch_1['sid_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  tokens_ = tokenized_raw.tolist()[0]

  # Padding
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  print("  check padding: ", np.array(padded).shape)

  # Masking
  attention_mask = np.where(padded != 0, 1, 0)
  print("  check mask:", attention_mask.shape)

  bertids = []
  # Implementing/Checking sentence merge
  df = df.assign(bertid=-1) 

  for s, _ in df.iterrows():
    dep_feats = df.loc[s].values.tolist() # labeled token and its metadata
    word = dep_feats[0].replace('-','').replace('.', '').replace('LRB', '[').replace('RRB', ']').lower() #.partition('/')[0]
    
    tok_index = np.array([True if word.startswith(elem) else False for x,elem in enumerate(tokens_)])
    bertid = np.argwhere(tok_index).ravel().tolist()

    if len(bertids):
      bertids_ = []
      #greater than minimum possible indices of previous pos
      print(word,bertids,bertid)
      bertid = list(filter(lambda x: x > max([min(elem) for elem in (bertids)]),bertid))
      for index,elem in enumerate(bertids): 
        #less than maximum possible indices of next pos
        # max_ = min(min(elem),max(bertid)) #min(list(itertools.chain.from_iterable(bertids[index:])))
        bertids_.append(list(filter(lambda x: x < max(bertid),elem)))
      bertids = bertids_.copy()

    #   bertids.append(temp)
    # else:
    bertids.append(bertid)

  bertids = [x[0] for x in bertids]; bertids.sort()
  print(bertids)
  # combs = list(itertools.product(*bertids))
  # bertids = []
  # for order in combs:
  #   order = list(order)
  #   if len(tokenized_raw) == len(set(order)): bertids = order

  bertids.sort(); print(bertids)
  df["bertid"] = bertids
  print("Get embeddings (this is the slow part seems)")
  # Querying the model
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  print(df)
  with torch.no_grad():
    last_hidden_states, hidden_states = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)

  nwords = hidden_states[0].shape[0]
  nsentidxs = hidden_states[0].shape[1]
  nfeats = hidden_states[0].shape[2]

  ### save costly output
  torch.save(hidden_states, "/Users/abhijit/IGRI/data/step25_hidden_states.pt")
  #torch.load("data/step25_hidden_states20201129rice_orig_filtering.pt")
  #torch.load("data/step25_hidden_states_20201129frey_less_filtering.pt)
  #torch.load("data/step25_hidden_states.pt")

  if False: # get last layer
    features = last_hidden_states[:,1,:].numpy()
  if False: # get last layer, by word
    mask = torch.tensor([[[v,]*nfeats,] for v in df.bertid])
    features = torch.gather(hidden_states[6][:,:,:], 1, mask)
    ### get to right dimensionality and type
    features  = features[:,0,:].numpy()
    ### tested picking out with this line
    ###  torch.gather(hidden_states[6][::50,:,0], 1, torch.tensor([[v%169,] for v in range(172)]))[-10:]
  if True: # get word embeddings for last four layers plus whole sentence embedding, last layer
    mask = torch.tensor([[[v,]*nfeats,] for v in df.bertid])
    features = np.concatenate((
                              hidden_states[6][:,0,:].numpy(),
                              torch.gather(hidden_states[6][:,:,:], 1, mask)[:,0,:].numpy(),
                              torch.gather(hidden_states[5][:,:,:], 1, mask)[:,0,:].numpy(),
                              torch.gather(hidden_states[4][:,:,:], 1, mask)[:,0,:].numpy(),
                              torch.gather(hidden_states[3][:,:,:], 1, mask)[:,0,:].numpy()
                            ),
                            axis=1)


  # sanity check model output format
  assert( np.all(hidden_states[-1].numpy() == last_hidden_states[:,:,:].numpy()) )
  labels = batch_1[1]
  all_features.append(features)

324
         0  sid                                           sid_text  tid  \
0   voting  324  ROOT the voting members of the board shall be ...    2   
1  members  324  ROOT the voting members of the board shall be ...    3   
2       of  324  ROOT the voting members of the board shall be ...    4   
3      the  324  ROOT the voting members of the board shall be ...    5   
4    board  324  ROOT the voting members of the board shall be ...    6   

           1  2  
0  Attribute  1  
1  Attribute  1  
2  Attribute  1  
3  Attribute  1  
4  Attribute  1  
Prep data
  check padding:  (18, 22)
  check mask: (18, 22)
members [[2]] [3]
of [[2], [3]] [4, 10, 15, 18]
the [[2], [3], [4, 10, 15, 18]] [1, 5, 16]
board [[2], [3], [4, 10, 15], [5, 16]] [6]
shall [[2], [3], [4], [5], [6]] [7]
be [[2], [3], [4], [5], [6], [7]] [8]
comprised [[2], [3], [4], [5], [6], [7], [8]] [9]
of [[2], [3], [4], [5], [6], [7], [8], [9]] [4, 10, 15, 18]
residents [[2], [3], [4], [5], [6], [7], [8], [9], [10, 15,

326
          0  sid                                           sid_text  tid  \
49    board  326  ROOT the board shall consist of five voting me...    2   
50    shall  326  ROOT the board shall consist of five voting me...    3   
51  consist  326  ROOT the board shall consist of five voting me...    4   
52       of  326  ROOT the board shall consist of five voting me...    5   
53     five  326  ROOT the board shall consist of five voting me...    6   

            1  2  
49  Attribute  1  
50    Deontic  3  
51        Aim  0  
52        Aim  0  
53     Object  4  
Prep data
  check padding:  (7, 11)
  check mask: (7, 11)
shall [[2]] [3]
consist [[2], [3]] [4]
of [[2], [3], [4]] [5]
five [[2], [3], [4], [5]] [6]
voting [[2], [3], [4], [5], [6]] [7]
members [[2], [3], [4], [5], [6], [7]] [8]
[2, 3, 4, 5, 6, 7, 8]
[2, 3, 4, 5, 6, 7, 8]
Get embeddings (this is the slow part seems)
          0  sid                                           sid_text  tid  \
49    board  326  root the boa

329
           0  sid                                           sid_text  tid  \
98     board  329  ROOT the board shall consist of three nonvotin...    2   
99     shall  329  ROOT the board shall consist of three nonvotin...    3   
100  consist  329  ROOT the board shall consist of three nonvotin...    4   
101       of  329  ROOT the board shall consist of three nonvotin...    5   
102    three  329  ROOT the board shall consist of three nonvotin...    6   

             1  2  
98   Attribute  1  
99     Deontic  3  
100        Aim  0  
101        Aim  0  
102     Object  4  
Prep data
  check padding:  (7, 13)
  check mask: (7, 13)
shall [[2]] [3]
consist [[2], [3]] [4]
of [[2], [3], [4]] [5]
three [[2], [3], [4], [5]] [6]
nonvoting [[2], [3], [4], [5], [6]] [7]
members [[2], [3], [4], [5], [6], [7]] [10]
[2, 3, 4, 5, 6, 7, 10]
[2, 3, 4, 5, 6, 7, 10]
Get embeddings (this is the slow part seems)
             0  sid                                           sid_text  tid  \
98      

333
               0  sid                                           sid_text  tid  \
141        board  333  ROOT the board chairperson shall be appointed ...    2   
142  chairperson  333  ROOT the board chairperson shall be appointed ...    3   
143        shall  333  ROOT the board chairperson shall be appointed ...    4   
144           be  333  ROOT the board chairperson shall be appointed ...    5   
145    appointed  333  ROOT the board chairperson shall be appointed ...    6   

             1  2  
141  Attribute  1  
142  Attribute  1  
143    Deontic  3  
144        Aim  0  
145        Aim  0  
Prep data
  check padding:  (10, 17)
  check mask: (10, 17)
chairperson [[2, 13]] [3]
shall [[2], [3]] [4]
be [[2], [3], [4]] [5]
appointed [[2], [3], [4], [5]] [6, 8]
by [[2], [3], [4], [5], [6, 8]] [7]
of [[2], [3], [4], [5], [6], [7]] [11]
the [[2], [3], [4], [5], [6], [7], [11]] [1, 12]
board [[2], [3], [4], [5], [6], [7], [11], [12]] [2, 13]
members [[2], [3], [4], [5], [6], [7], [

336
           0  sid                                           sid_text  tid  \
185      the  336  ROOT the terms of the initial appointments sha...    1   
186    terms  336  ROOT the terms of the initial appointments sha...    2   
187       of  336  ROOT the terms of the initial appointments sha...    3   
188      the  336  ROOT the terms of the initial appointments sha...    4   
189  initial  336  ROOT the terms of the initial appointments sha...    5   

          1  2  
185  Object  4  
186  Object  4  
187  Object  4  
188  Object  4  
189  Object  4  
Prep data
  check padding:  (11, 15)
  check mask: (11, 15)
terms [[1, 4]] [2]
of [[1], [2]] [3]
the [[1], [2], [3]] [1, 4]
initial [[1], [2], [3], [4]] [5]
appointments [[1], [2], [3], [4], [5]] [6]
shall [[1], [2], [3], [4], [5], [6]] [7]
be [[1], [2], [3], [4], [5], [6], [7]] [8]
designated [[1], [2], [3], [4], [5], [6], [7], [8]] [9]
city [[1], [2], [3], [4], [5], [6], [7], [8], [9]] [11]
council [[1], [2], [3], [4], [5], [

...

864
            0  sid                                           sid_text  tid  \
9257   copies  864  ROOT copies of all minutes , motions , resolut...    1   
9258       of  864  ROOT copies of all minutes , motions , resolut...    2   
9259      all  864  ROOT copies of all minutes , motions , resolut...    3   
9260  minutes  864  ROOT copies of all minutes , motions , resolut...    4   
9261  motions  864  ROOT copies of all minutes , motions , resolut...    6   

           1  2  
9257  Object  4  
9258  Object  4  
9259  Object  4  
9260  Object  4  
9261  Object  4  
Prep data
  check padding:  (17, 23)
  check mask: (17, 23)
of [[1]] [2]
all [[1], [2]] [3]
minutes [[1], [2], [3]] [4]
motions [[1], [2], [3], [4]] [6]
resolutions [[1], [2], [3], [4], [6]] [8]
findings [[1], [2], [3], [4], [6], [8]] [10]
and [[1], [2], [3], [4], [6], [8], [10]] [11]
reports [[1], [2], [3], [4], [6], [8], [10], [11]] [12]
shall [[1], [2], [3], [4], [6], [8], [10], [11], [12]] [13]
be [[1], [2], [3]

866
              0  sid                                           sid_text  tid  \
9322     review  866  ROOT review and recommend as provided in secti...    1   
9323        and  866  ROOT review and recommend as provided in secti...    2   
9324  recommend  866  ROOT review and recommend as provided in secti...    3   
9325         as  866  ROOT review and recommend as provided in secti...    4   
9326   provided  866  ROOT review and recommend as provided in secti...    5   

              1  2  
9322  Condition  2  
9323  Condition  2  
9324  Condition  2  
9325  Condition  2  
9326  Condition  2  
Prep data
  check padding:  (37, 48)
  check mask: (37, 48)
and [[1]] [2, 13, 29, 32]
recommend [[1], [2, 13, 29, 32]] [3]
as [[1], [2], [3]] [4]
provided [[1], [2], [3], [4]] [5]
in [[1], [2], [3], [4], [5]] [6]
section [[1], [2], [3], [4], [5], [6]] [7, 20, 39]
006 [[1], [2], [3], [4], [5], [6], [7, 20, 39]] [8]
the [[1], [2], [3], [4], [5], [6], [7], [8]] [10, 17, 30, 36]
proposed [[

868
            0  sid                                           sid_text  tid  \
9359   review  868  ROOT review and comment on the pertinent porti...    1   
9360      and  868  ROOT review and comment on the pertinent porti...    2   
9361  comment  868  ROOT review and comment on the pertinent porti...    3   
9362       on  868  ROOT review and comment on the pertinent porti...    4   
9363      the  868  ROOT review and comment on the pertinent porti...    5   

              1  2  
9359  Condition  2  
9360  Condition  2  
9361  Condition  2  
9362  Condition  2  
9363  Condition  2  
Prep data
  check padding:  (32, 41)
  check mask: (32, 41)
and [[1]] [2, 15, 32, 35]
comment [[1], [2, 15, 32, 35]] [3]
on [[1], [2], [3]] [4]
the [[1], [2], [3], [4]] [5, 11, 23]
pertinent [[1], [2], [3], [4], [5, 11, 23]] [6]
portions [[1], [2], [3], [4], [5], [6]] [9]
of [[1], [2], [3], [4], [5], [6], [9]] [10, 22, 25, 30]
the [[1], [2], [3], [4], [5], [6], [9], [10, 22, 25, 30]] [5, 11, 23]
pr

872
                0  sid                                           sid_text  \
9413  participate  872  ROOT participate in the review and comment on ...   
9414           in  872  ROOT participate in the review and comment on ...   
9415          the  872  ROOT participate in the review and comment on ...   
9416       review  872  ROOT participate in the review and comment on ...   
9417          and  872  ROOT participate in the review and comment on ...   

      tid          1  2  
9413    1  Condition  2  
9414    2  Condition  2  
9415    3  Condition  2  
9416    4  Condition  2  
9417    5  Condition  2  
Prep data
  check padding:  (20, 28)
  check mask: (20, 28)
in [[1]] [2]
the [[1], [2]] [3, 11]
review [[1], [2], [3, 11]] [4]
and [[1], [2], [3], [4]] [5, 23]
comment [[1], [2], [3], [4], [5, 23]] [6]
on [[1], [2], [3], [4], [5], [6]] [7]
any [[1], [2], [3], [4], [5], [6], [7]] [8]
project [[1], [2], [3], [4], [5], [6], [7], [8]] [9]
affecting [[1], [2], [3], [4], [5], [6],

878
            0  sid                                           sid_text  tid  \
9453  prepare  878  ROOT prepare and present an annual state of fo...    1   
9454      and  878  ROOT prepare and present an annual state of fo...    2   
9455  present  878  ROOT prepare and present an annual state of fo...    3   
9456       an  878  ROOT prepare and present an annual state of fo...    4   
9457   annual  878  ROOT prepare and present an annual state of fo...    5   

              1  2  
9453  Condition  2  
9454  Condition  2  
9455  Condition  2  
9456  Condition  2  
9457  Condition  2  
Prep data
  check padding:  (19, 22)
  check mask: (19, 22)
and [[1]] [2, 4, 10, 17]
present [[1], [2, 4, 10, 17]] [3]
an [[1], [2], [3]] [4]
annual [[1], [2], [3], [4]] [4, 5]
state [[1], [2], [3], [4], [5]] [6]
of [[1], [2], [3], [4], [5], [6]] [7]
food [[1], [2], [3], [4], [5], [6], [7]] [8]
access [[1], [2], [3], [4], [5], [6], [7], [8]] [9]
and [[1], [2], [3], [4], [5], [6], [7], [8], [9]] [2,

In [21]:
#all_features = np.concatenate(all_features,axis=0);print("Embeds produced: ",all_features.shape)
np.savetxt("/Users/abhijit/IGRI/data/step1_embedding_features.csv", all_features, delimiter=",", header=",".join(map(str,range(features.shape[1]))))


Embeds produced:  (36464640,)
